# Direct Marketing with Amazon SageMaker Autopilot

This notebook works well with the `Python 3 (Data Science)` kernel on SageMaker Studio.

---

---

## Contents

1. [Introduction](#Introduction)
1. [Prerequisites](#Prerequisites)
1. [Downloading the dataset](#Downloading)
1. [Upload the dataset to Amazon S3](#Uploading)

## Introduction


This cnotebook must be completed before other exercises of Workshop 2. In this workshop, we load the training data to be used by the Autopilot experiment to an S3 bucket.
Execute each cell in this notebook by pressing `Shift + Enter`.



## Prerequisites

In this first cell, we'll:

- **configure** the S3 bucket and folder where data should be stored (to keep our environment tidy)
- **connect** to SageMaker with the open-source [Sagemaker Python SDK](https://sagemaker.readthedocs.io/en/stable/), `sagemaker`

Note that while [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) is the general-purpose AWS SDK for Python, `sagemaker` provides some powerful, higher-level interfaces designed specifically for ML workflows.

In [5]:
import sagemaker

session = sagemaker.Session()

bucket = session.default_bucket()
prefix = "mlu-workshop/autopilot-dm"

sm = session.sagemaker_client

role = sagemaker.get_execution_role()

## Downloading the dataset<a name="Downloading"></a>

In this example we'll use the **direct marketing dataset** as per:

> *\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014*

Here, we'll download [the dataset](https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip) from the SageMaker sample data S3 bucket.

In [2]:
!wget -P data/ -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

import zipfile

with zipfile.ZipFile("data/bank-additional.zip", "r") as zip_ref:
    print("Unzipping...")
    zip_ref.extractall("data")
print("Done")

local_data_path = "./data/bank-additional/bank-additional-full.csv"

--2022-02-19 04:35:12--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.218.184.81
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.218.184.81|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘data/bank-additional.zip’ not modified on server. Omitting download.

Unzipping...
Done


## Upload the dataset to Amazon S3<a name="Uploading"></a>

Before you run Autopilot on the dataset, first perform a check of the dataset to make sure that it has no obvious errors. The Autopilot process can take long time, and it's generally a good practice to inspect the dataset before you start a job. This particular dataset is small, so you can inspect it in the notebook instance itself. If you have a larger dataset that will not fit in a notebook instance memory, inspect the dataset offline using a big data analytics tool like Apache Spark. [Deequ](https://github.com/awslabs/deequ) is a library built on top of Apache Spark that can be helpful for performing checks on large datasets. Autopilot is capable of handling datasets up to 5 GB.

Read the data into a Pandas data frame and take a look.

In [6]:
import pandas as pd

data = pd.read_csv(local_data_path)
with pd.option_context("display.max_columns", 500):
    # Make sure we can see all of the columns
    display(data)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


Note that there are 20 features to help predict the target column 'y'.

Amazon SageMaker Autopilot takes care of preprocessing your data for you. You do not need to perform conventional data preprocssing techniques such as handling missing values, converting categorical features to numeric features, scaling data, and handling more complicated data types.

Moreover, splitting the dataset into training and validation splits is not necessary. Autopilot takes care of this for you. You may, however, want to split out a test set. That's next, although you use it for batch inference at the end instead of testing the model.


### Reserve some data for calling batch inference on the model

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split is reserved to perform inference using the suggested model.


In [7]:
train_data = data.sample(frac=0.8, random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=["y"])

#### Upload the prepared training dataset to Amazon S3 for the Autopilot experiment
Copy the file to Amazon Simple Storage Service (Amazon S3) in a .csv format for Amazon SageMaker training to use.

**Please note down the S3 object URI of variable `train_data_s3_path`, which will be used in doing AutoPilot job in SageMaker Studio UI**

In [10]:
train_file = "data/train_data.csv"
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print("Train data uploaded to: " + train_data_s3_path)

test_file = "data/test_data_without_label.csv"
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_without_label_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print("Test data without label uploaded to: " + test_data_without_label_s3_path)

test_file_label = "data/test_data_with_label.csv"
test_data.to_csv(test_file_label, index=False, header=True)
# test_data_with_label_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
# print("Test data uploaded to: " + test_data_with_label_s3_path)

Train data uploaded to: s3://sagemaker-us-west-2-783193689466/mlu-workshop/autopilot-dm/train/train_data.csv
Test data without label uploaded to: s3://sagemaker-us-west-2-783193689466/mlu-workshop/autopilot-dm/test/test_data_without_label.csv


### Store the shared variables

In [12]:
%store bucket
%store prefix
%store train_data_s3_path
%store test_file_label
#%store test_data_s3_path

Stored 'bucket' (str)
Stored 'prefix' (str)
Stored 'train_data_s3_path' (str)
Stored 'test_file_label' (str)


### Next

Since the data is sotred on S3 bucket, we are ready to kick off Autopilot experiment. Choose <b> only one</b> of the options below to create an Autopilot experiment.

a. [Using SageMaker Studio UI](./README.md#Using-SageMaker-Studio-UI)
 
b. [Using SageMaker Python SDK](./README.md#Using-Python-SDK)

a. [Using SageMaker Studio UI](./02a_sagemaker_autopilot_experiment_with_studio_ui.ipynb)
 
b. [Using SageMaker Python SDK](./02b_sagemaker_autopilot_experiment_with_sdk.ipynb)